In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import keras
import pandas
from enum import Enum
import json
import random
import cv2
from scipy.misc import toimage
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

# import utils

In [ ]:
np.random.seed(100)
train_file = 'data/train/train.json'
test_file = 'data/test/test.json'
IMG_HEIGHT = 75
IMG_WIDTH = 75

In [ ]:
def grayscale_to_rgb(img):
    return cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

In [ ]:
train_data = pd.read_json(train_file)
train_data.inc_angle = train_data.inc_angle.replace('na', 0)
train_data.inc_angle = train_data.inc_angle.astype(float).fillna(0.0)

In [ ]:
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_data["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train_data.inc_angle)
y_train = np.array(train_data["is_iceberg"])

In [ ]:
# X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
#                     , X_angle_train, y_train, random_state=123, train_size=0.85)

In [ ]:
optimizer = keras.optimizers.Nadam()

In [ ]:
def build_model(optimizer, input_shape):
    input_img = Input(shape=input_shape, name="img")
#     input_angle = Input(shape=(1), name="angle")
    activation = 'relu'
    x = Conv2D(32, (3,3), activation=activation)(input_img)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3,3), activation=activation)(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
    x = Conv2D(128, (3,3), activation=activation)(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
#     x = GlobalMaxPooling2D()(x)
#     x = Conv2D(256, (3,3), activation=activation)(x)
    x = Flatten()(x)
    x = Dense(1024, activation=activation)(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation=activation)(x)
    output = Dense(1, activation="sigmoid")(x)
    
    model = Model([input_img], output)
    model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    return model

model = build_model(optimizer, (IMG_WIDTH, IMG_HEIGHT, 3))
    

In [ ]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    

In [ ]:
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=3)
batch_size=32
model.fit(x=X_train, y=y_train, batch_size=batch_size,
#                     validation_split=0.2,
                    epochs=20,
                    callbacks=callbacks
                   )

# model.fit(X_train, y_train, epochs=25,
#         validation_split=0.15, batch_size=32
#          , callbacks=callbacks)